# Fixing `air_pollution_score` Data Type
- 2008: convert string to float
- 2018: convert int to float

In [1]:
# load dataset
import pandas as pd

df_08 = pd.read_csv('data_08.csv')
df_18 = pd.read_csv('data_18.csv')

In [2]:
# try using Pandas to_numeric or astype function to convert the
# 2008 air_pollution_score column to float -- this won't work

df_08['air_pollution_score'] = df_08['air_pollution_score'].astype('float64')
# df_08['air_pollution_score'].to_numeric(float64)

# Figuring out the issue
Looks like this isn't going to be as simple as converting the datatype. According to the error above, the value at row 582 is "6/4" - let's check it out.

In [3]:
df_08.iloc[582]

model                   MERCEDES-BENZ CL550
displ                                   5.5
cyl                                       8
trans                               Auto-L7
drive                                   2WD
fuel                               Gasoline
veh_class                         small car
air_pollution_score                       7
city_mpg                                 14
hwy_mpg                                  21
cmb_mpg                                  17
greenhouse_gas_score                      4
smartway                                 no
Name: 582, dtype: object

# It's not just the air pollution score!
The mpg columns and greenhouse gas scores also seem to have the same problem - maybe that's why these were all saved as strings! According to [this link](http://www.fueleconomy.gov/feg/findacarhelp.shtml#airPollutionScore), which I found from the PDF documentation:

    "If a vehicle can operate on more than one type of fuel, an estimate is provided for each fuel type."
    
Ohh.. so all vehicles with more than one fuel type, or hybrids, like the one above (it uses ethanol AND gas) will have a string that holds two values - one for each. This is a little tricky, so I'm going to show you how to do it with the 2008 dataset, and then you'll try it with the 2018 dataset.

In [4]:
# First, let's get all the hybrids in 2008
hb_08 = df_08[df_08['fuel'].str.contains('/')]
hb_08

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


Looks like this dataset only has one! The 2018 has MANY more - but don't worry - the steps I'm taking here will work for that as well!

In [5]:
# hybrids in 2018
hb_18 = df_18[df_18['fuel'].str.contains('/')]
hb_18

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


We're going to take each hybrid row and split them into two new rows - one with values for the first fuel type (values before the "/"), and the other with values for the second fuel type (values after the "/"). Let's separate them with two dataframes!

In [6]:
# create two copies of the 2008 hybrids dataframe
df1 = hb_08.copy()  # data on first fuel type of each hybrid vehicle
df2 = hb_08.copy()  # data on second fuel type of each hybrid vehicle

# Each one should look like this
df1

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


For this next part, we're going use Pandas' apply function. See the docs [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

In [7]:
# columns to split by "/"
split_columns = ['fuel', 'air_pollution_score', 'city_mpg', 'hwy_mpg', 'cmb_mpg', 'greenhouse_gas_score']

# apply split function to each column of each dataframe copy
for c in split_columns:
    df1[c] = df1[c].apply(lambda x: x.split("/")[0])
    df2[c] = df2[c].apply(lambda x: x.split("/")[1])

In [8]:
# this dataframe holds info for the FIRST fuel type of the hybrid
# aka the values before the "/"s
df1

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [9]:
# this dataframe holds info for the SECOND fuel type of the hybrid
# aka the values before the "/"s
df2

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [10]:
# combine dataframes to add to the original dataframe
new_rows = df1.append(df2)

# now we have separate rows for each fuel type of each vehicle!
new_rows

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [11]:
# drop the original hybrid rows
df_08.drop(hb_08.index, inplace=True)

# add in our newly separated rows
df_08 = df_08.append(new_rows, ignore_index=True)

In [12]:
# check that all the original hybrid rows with "/"s are gone
df_08[df_08['fuel'].str.contains('/')]

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [13]:
df_08.shape

(987, 13)

# Repeat this process for the 2018 dataset

In [14]:
# create two copies of the 2018 hybrids dataframe, hb_18
df1 = hb_18.copy()
df2 = hb_18.copy()

### Split values for `fuel`, `city_mpg`, `hwy_mpg`, `cmb_mpg`
You don't need to split for `air_pollution_score` or `greenhouse_gas_score` here because these columns are already ints in the 2018 dataset.

In [15]:
# list of columns to split
split_columns = ['fuel', 'city_mpg', 'hwy_mpg', 'cmb_mpg']

# apply split function to each column of each dataframe copy
for c in split_columns:
    df1[c] = df1[c].apply(lambda x: x.split("/")[0])
    df2[c] = df2[c].apply(lambda x: x.split('/')[1])

In [16]:
# append the two dataframes
new_rows = df1.append(df2)

# drop each hybrid row from the original 2018 dataframe
# do this by using Pandas drop function with hb_18's index
df_18.drop(hb_18.index, inplace=True)

# append new_rows to df_18
df_18 = df_18.append(new_rows, ignore_index=True)

In [17]:
# check that they're gone
df_18[df_18['fuel'].str.contains('/')]

,model,displ,cyl,trans,drive,fuel,veh_class,air_pollution_score,city_mpg,hwy_mpg,cmb_mpg,greenhouse_gas_score,smartway


In [18]:
df_18.shape

(832, 13)

In [19]:
df_08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987 entries, 0 to 986
Data columns (total 13 columns):
model                   987 non-null object
displ                   987 non-null float64
cyl                     987 non-null int64
trans                   987 non-null object
drive                   987 non-null object
fuel                    987 non-null object
veh_class               987 non-null object
air_pollution_score     987 non-null float64
city_mpg                987 non-null int64
hwy_mpg                 987 non-null int64
cmb_mpg                 987 non-null int64
greenhouse_gas_score    987 non-null int64
smartway                987 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 100.3+ KB


### Now we can comfortably continue the changes needed for `air_pollution_score`! Here they are again:
- 2008: convert string to float
- 2018: convert int to float

In [20]:
# convert string to float for 2008 air pollution column
df_08['air_pollution_score'].astype('float64')

0      7.0
1      7.0
2      7.0
3      7.0
4      7.0
5      7.0
6      6.0
7      6.0
8      6.0
9      6.0
10     6.0
11     6.0
12     6.0
13     6.0
14     7.0
15     7.0
16     6.0
17     7.0
18     6.0
19     6.0
20     6.0
21     6.0
22     6.0
23     6.0
24     6.0
25     6.0
26     6.0
27     6.0
28     6.0
29     6.0
      ... 
957    7.0
958    7.0
959    7.0
960    7.0
961    7.0
962    7.0
963    7.0
964    7.0
965    7.0
966    7.0
967    6.0
968    6.0
969    7.0
970    7.0
971    7.0
972    7.0
973    9.5
974    7.0
975    7.0
976    7.0
977    7.0
978    7.0
979    7.0
980    7.0
981    7.0
982    7.0
983    7.0
984    7.0
985    6.0
986    4.0
Name: air_pollution_score, Length: 987, dtype: float64

In [21]:
# convert int to float for 2018 air pollution column
df_18['air_pollution_score'].astype('float64')

0      3.0
1      3.0
2      3.0
3      3.0
4      3.0
5      3.0
6      3.0
7      1.0
8      3.0
9      7.0
10     7.0
11     7.0
12     7.0
13     5.0
14     3.0
15     3.0
16     5.0
17     3.0
18     5.0
19     5.0
20     5.0
21     5.0
22     3.0
23     3.0
24     3.0
25     3.0
26     3.0
27     3.0
28     3.0
29     3.0
      ... 
802    3.0
803    3.0
804    3.0
805    3.0
806    3.0
807    7.0
808    3.0
809    3.0
810    3.0
811    3.0
812    7.0
813    5.0
814    5.0
815    3.0
816    3.0
817    3.0
818    3.0
819    3.0
820    3.0
821    3.0
822    1.0
823    5.0
824    5.0
825    3.0
826    3.0
827    5.0
828    5.0
829    7.0
830    7.0
831    7.0
Name: air_pollution_score, Length: 832, dtype: float64

In [22]:
df_08.to_csv('data_08.csv', index=False)
df_18.to_csv('data_18.csv', index=False)